In [1]:
# TRANSFER LEARNING FINE TUNING -  MASS BENIGN MALIGNANT CLASSIFIER USING BCDR MAMMOGRAM PATCHES
# No CW
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
%matplotlib inline
tensorflow.keras.backend.set_image_data_format('channels_last')
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.layers import ReLU
import time


vid='v004'

In [2]:
# Create CNN

IMG_SIZE=331

from tensorflow.keras.applications.nasnet import NASNetLarge
base_model=NASNetLarge(weights='imagenet',include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))

for layer in base_model.layers:
    layer.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(256)(x)
x=ReLU()(x)
x=Dropout(0.5)(x)
x=Dense(128)(x) 
x=ReLU()(x)
preds=Dense(1,activation='sigmoid')(x)

model=Model(inputs=base_model.input,outputs=preds)

In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 331, 331, 3) 0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 165, 165, 96) 2592        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 165, 165, 96) 384         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation (Activation)         (None, 165, 165, 96) 0           stem_bn1[0][0]                   
______________________________________________________________________________________________

separable_conv_1_normal_left5_9 (None, 21, 21, 336)  115920      activation_138[0][0]             
__________________________________________________________________________________________________
separable_conv_1_bn_normal_left (None, 21, 21, 336)  1344        separable_conv_1_normal_left1_9[0
__________________________________________________________________________________________________
separable_conv_1_bn_normal_righ (None, 21, 21, 336)  1344        separable_conv_1_normal_right1_9[
__________________________________________________________________________________________________
separable_conv_1_bn_normal_left (None, 21, 21, 336)  1344        separable_conv_1_normal_left2_9[0
__________________________________________________________________________________________________
separable_conv_1_bn_normal_righ (None, 21, 21, 336)  1344        separable_conv_1_normal_right2_9[
__________________________________________________________________________________________________
separable_

separable_conv_2_bn_normal_left (None, 11, 11, 672)  2688        separable_conv_2_normal_left2_15[
__________________________________________________________________________________________________
separable_conv_2_bn_normal_righ (None, 11, 11, 672)  2688        separable_conv_2_normal_right2_15
__________________________________________________________________________________________________
normal_left3_15 (AveragePooling (None, 11, 11, 672)  0           normal_bn_1_15[0][0]             
__________________________________________________________________________________________________
normal_left4_15 (AveragePooling (None, 11, 11, 672)  0           adjust_bn_15[0][0]               
__________________________________________________________________________________________________
normal_right4_15 (AveragePoolin (None, 11, 11, 672)  0           adjust_bn_15[0][0]               
__________________________________________________________________________________________________
separable_

In [4]:
len(model.layers)

1046

In [5]:
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy']) 

In [6]:
# Image preprocessing and data augmentation

batch_size=4

train_datagen = ImageDataGenerator(rescale=1./255,
                         horizontal_flip = True,
                         vertical_flip = True,
                         fill_mode="wrap",
                         width_shift_range = 0.2,
                         height_shift_range = 0.2,
                         zoom_range = 0.2,
                         rotation_range = 90,
                         shear_range=0.2
                        )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory('mass_ben_mal_class01/train01', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)
valid_generator = valid_datagen.flow_from_directory('mass_ben_mal_class01/valid01', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)


Found 532 images belonging to 2 classes.
Found 134 images belonging to 2 classes.


In [7]:
#Callbacks
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=2, min_delta=1e-4, mode='min')
    early_stop_cr=EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=2, mode='auto', restore_best_weights=True)
    return [mcp_save, reduce_lr_loss, early_stop_cr]


In [8]:
#Training stage 1
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

name_weights = "mass_ben_mal_class_nasnetLarge_weights_"+vid+".h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=3,
                   verbose=1,
                   callbacks = callbacks)  

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
133/133 [==============================] - 92s 356ms/step - loss: 0.6023 - accuracy: 0.6917 - val_loss: 0.5436 - val_accuracy: 0.7273


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/3
133/133 [==============================] - 38s 283ms/step - loss: 0.5418 - accuracy: 0.7425 - val_loss: 0.3534 - val_accuracy: 0.8409
Epoch 3/3
133/133 [==============================] - 37s 280ms/step - loss: 0.4728 - accuracy: 0.7801 - val_loss: 0.3413 - val_accuracy: 0.8636


In [9]:
#Training stage 2
for layer in model.layers[:700]:
    layer.trainable=False
for layer in model.layers[700:]:  #last ~33% layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=10,
                   verbose=1,
                   callbacks = callbacks)

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10
133/133 [==============================] - 110s 490ms/step - loss: 0.5111 - accuracy: 0.7594 - val_loss: 0.3535 - val_accuracy: 0.8788
Epoch 2/10
133/133 [==============================] - 55s 416ms/step - loss: 0.4301 - accuracy: 0.8252 - val_loss: 0.4124 - val_accuracy: 0.8561
Epoch 3/10
133/133 [==============================] - 55s 416ms/step - loss: 0.3663 - accuracy: 0.8534 - val_loss: 0.3417 - val_accuracy: 0.8864
Epoch 4/10
133/133 [==============================] - 55s 415ms/step - loss: 0.3976 - accuracy: 0.8609 - val_loss: 0.3254 - val_accuracy: 0.8788
Epoch 5/10
133/133 [==============================] - 55s 416ms/step - loss: 0.3558 - accuracy: 0.8590 - val_loss: 0.3196 - val_accuracy: 0.8864
Epoch 6/10
133/133 [==============================] - 55s 414ms/step - loss: 0.3500 - accuracy: 0.8722 - val_loss: 0.2633 - val_accuracy: 0.8712
Epoch 7/10
133/133 [==============================] - 55s 417ms/step - loss: 0.3032 - accuracy: 0.8722 - val_loss: 0.4064 - val_a

In [10]:
#Training stage 3
#time.sleep(360)
for layer in model.layers:  #All layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
callbacks = get_callbacks(name_weights = name_weights, patience_lr=5)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=150,
                   verbose=1,
                   callbacks = callbacks)

Epoch 1/150
133/133 [==============================] - 189s 929ms/step - loss: 0.3365 - accuracy: 0.8703 - val_loss: 0.4789 - val_accuracy: 0.8712
Epoch 2/150
133/133 [==============================] - 113s 853ms/step - loss: 0.3055 - accuracy: 0.8778 - val_loss: 0.4670 - val_accuracy: 0.8712
Epoch 3/150
133/133 [==============================] - 114s 856ms/step - loss: 0.3125 - accuracy: 0.8853 - val_loss: 0.3851 - val_accuracy: 0.8712
Epoch 4/150
133/133 [==============================] - 114s 857ms/step - loss: 0.2925 - accuracy: 0.8816 - val_loss: 0.4889 - val_accuracy: 0.8636
Epoch 5/150
133/133 [==============================] - 119s 894ms/step - loss: 0.2852 - accuracy: 0.8891 - val_loss: 0.4628 - val_accuracy: 0.8788
Epoch 6/150
133/133 [==============================] - 122s 918ms/step - loss: 0.2295 - accuracy: 0.9135 - val_loss: 0.4730 - val_accuracy: 0.8712
Epoch 7/150
133/133 [==============================] - 118s 888ms/step - loss: 0.2643 - accuracy: 0.9079 - val_loss: 0

In [11]:
model.metrics_names

['loss', 'accuracy']

In [12]:
# Validation accuracy, validation data confusion matrix, area under the roc score
valid_generator = valid_datagen.flow_from_directory('mass_ben_mal_class01/valid01', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)

# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(valid_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(valid_generator.classes, Y_pred)
prec=precision_score(valid_generator.classes, y_pred,pos_label=1)
rec=recall_score(valid_generator.classes, y_pred,pos_label=1)
f1s=f1_score(valid_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(valid_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(valid_generator.classes, y_pred))
print('Accuracy:', accuracy_score(valid_generator.classes, y_pred))
print('ROC AUC score:', ras)
print('Precision score:', prec)
print('Recall score:', rec)
print('F1 score:', f1s)
print('MCC score:', mcc)

#Print estimation result for ensemble model
print(Y_pred)
print(valid_generator.classes)

Found 134 images belonging to 2 classes.


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Confusion Matrix:
[[82  9]
 [ 9 34]]
Accuracy: 0.8656716417910447
ROC AUC score: 0.946843853820598
Precision score: 0.7906976744186046
Recall score: 0.7906976744186046
F1 score: 0.7906976744186046
MCC score: 0.6917965755175057
[[1.15116673e-05]
 [8.89920106e-04]
 [1.20897776e-05]
 [2.20789411e-03]
 [1.31380220e-04]
 [9.86952066e-01]
 [7.35253270e-05]
 [9.73553240e-01]
 [6.18293788e-03]
 [1.01731011e-05]
 [8.49523119e-08]
 [1.68659317e-03]
 [7.09515929e-01]
 [2.67990595e-06]
 [7.97949612e-01]
 [1.99173111e-03]
 [1.84646592e-06]
 [1.36726191e-02]
 [1.10280116e-05]
 [2.73474329e-03]
 [1.45980599e-03]
 [6.03649840e-02]
 [3.65088526e-05]
 [4.50918451e-03]
 [3.61906859e-06]
 [6.11390769e-02]
 [2.10916813e-04]
 [1.33863607e-04]
 [6.07265210e-05]
 [5.64897759e-03]
 [2.03165328e-05]
 [1.07358165e-01]
 [7.20345527e-02]
 [2.63088886e-02]
 [2.44491646e-04]
 [2.49502609e-05]
 [3.70766558e-02]
 [1.53227156e-04]
 [3.02972751e-07]
 [4.77560498e-05]
 [9.23760876e-04]
 [2.58377707e-03]
 [1.01011365e-05]

In [13]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('mass_ben_mal_class01/test', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)


# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(test_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(test_generator.classes, Y_pred)
prec=precision_score(test_generator.classes, y_pred,pos_label=1)
rec=recall_score(test_generator.classes, y_pred,pos_label=1)
f1s=f1_score(test_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(test_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(test_generator.classes, y_pred))
print('Accuracy:', accuracy_score(test_generator.classes, y_pred))
print('ROC AUC score:', ras)
print('Precision score:', prec)
print('Recall score:', rec)
print('F1 score:', f1s)
print('MCC score:', mcc)



Found 169 images belonging to 2 classes.


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Confusion Matrix:
[[101  12]
 [ 23  33]]
Accuracy: 0.7928994082840237
ROC AUC score: 0.8323324905183312
Precision score: 0.7333333333333333
Recall score: 0.5892857142857143
F1 score: 0.6534653465346534
MCC score: 0.5144522891222515


In [15]:
#Print estimation result for ensemble model
print(Y_pred)
print(test_generator.classes)

[[7.66479783e-03]
 [2.90066265e-02]
 [1.63336527e-02]
 [2.30871170e-04]
 [1.86971607e-04]
 [5.98974111e-06]
 [6.92226862e-08]
 [3.85171406e-05]
 [6.66233886e-04]
 [7.02137302e-04]
 [8.40670196e-04]
 [5.18395755e-05]
 [8.44688384e-06]
 [1.09881430e-03]
 [1.01671077e-03]
 [9.96815697e-09]
 [5.19306898e-01]
 [2.80224800e-01]
 [3.04013270e-07]
 [3.24163283e-03]
 [6.64647203e-03]
 [1.04810824e-05]
 [2.20858874e-05]
 [8.36884737e-01]
 [4.71459982e-09]
 [1.21115882e-08]
 [5.33170635e-07]
 [1.12627237e-03]
 [4.31749672e-01]
 [1.42281991e-04]
 [3.93449329e-04]
 [5.12103364e-03]
 [1.60707423e-05]
 [6.27235711e-01]
 [4.86402176e-02]
 [2.41161746e-04]
 [1.24553553e-04]
 [8.77269685e-01]
 [4.85020131e-03]
 [9.98928607e-01]
 [1.17344107e-03]
 [2.22773997e-06]
 [6.78029552e-04]
 [5.29173485e-05]
 [4.88038689e-01]
 [6.97839350e-05]
 [3.46759509e-04]
 [7.09516215e-08]
 [9.98705029e-01]
 [1.32188388e-05]
 [2.05245897e-06]
 [6.18919078e-03]
 [1.33029253e-05]
 [7.07927938e-08]
 [7.05155827e-08]
 [9.804335